In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

def create_thin_flag(df):
    at20 = df.AT20.copy()
    df['thin_flag'] = (np.where(at20 >= 0, 1, 0)) & (np.where(at20 <= 24, 1, 0))
    return df


def create_prior_dist_bk_flag(df):
    ftb16 = df.FTB16.copy()
    df['prior_dist_BK_flag'] = (np.where(ftb16 >= 0, 1, 0)) & (np.where(ftb16 <= 48, 1, 0))
    return df


def create_features(df):
    df = (df
          .pipe(create_thin_flag)
          .pipe(create_prior_dist_bk_flag)
          .rename(columns={'Vantage_Score_3': 'VTG03TOT', 'EADS51_Score': 'EADS51'}))
    return df


def bc31_at35_cut(df):
    df = df.copy()
    df['BC31_AT35_cut']= df.apply(lambda x: 'Cut' if ((x.BC31>14.85) & (x.AT35<=2775.5)) else 'Keep', axis=1)
    return df


def bc31_at35_bc21_cut(df):
    df = df.copy()
    df['BC31_AT35_BC21_cut']= df.apply(lambda x: 'Cut' if ((x.BC31>14.85) & (x.AT35>2775.5) & (x.BC21<=8.5)) else 'Keep', axis=1)
    return df

def apply_all_cuts(df):
    df = df.copy()
    df['all_cuts']= df.apply(lambda x: 'Cut' if (x.BC31_AT35_cut == 'Cut') | (x.BC31_AT35_BC21_cut == 'Cut') else 'Keep', axis=1)
    return df

def decision(df):
    if df['BC31_AT35_cut'] == 'Cut':
        market_cell = 99
    elif df['BC31_AT35_BC21_cut'] == 'Cut':
        market_cell= 98
    elif  df['all_cuts'] == 'Keep':
      if 660 <= df['CEMP09'] <= 679:
        if 0.015652 < df['scored_trio_uncalibrated'] <= 1:
                market_cell = 13 # TRIO $100/$500
        elif 0.003889 < df['scored_ts_uncalibrated'] <= 1:
                market_cell = 14 # Truly Simple 15m
        else:
                market_cell = None

      # FICO 680-699
      elif 680 <= df['CEMP09'] <= 699:
        if 0.015652 < df['scored_trio_uncalibrated'] <= 1:
            market_cell = 15 # TRIO $100/$500
        elif 0.003889 < df['scored_ts_uncalibrated'] <= 1:
            market_cell = 16 # Truly Simple 15m
        else:
            market_cell = None

      # FICO 700-719
      elif 700 <= df['CEMP09'] <= 719:
        if 0.015652 < df['scored_trio_uncalibrated'] <= 1:
            market_cell = 17 # TRIO $100/$500
        elif 0.001903 < df['scored_ts_uncalibrated'] <= 1:
            market_cell = 18 # Truly Simple 15m
        else:
            market_cell = None

    # FICO 720-750
      elif 720 <= df['CEMP09'] <= 749:
        if 0.003889 < df['scored_ts_uncalibrated'] <= 1:
            market_cell = 19 # Truly Simple 15m
        else:
            market_cell = None

    # FICO 750-799
      elif 750 <= df['CEMP09'] <= 799:
        if 0.003889 < df['scored_ts_uncalibrated'] <= 1:
            market_cell = 20 # Truly Simple 15m
        else:
            market_cell = None

    # FICO 800-850
      elif 800 <= df['CEMP09'] <= 850:
        if 0.003889 < df['scored_ts_uncalibrated'] <= 1:
            market_cell = 21 # Truly Simple 15m
        else:
            market_cell = None
      else:
        market_cell = None
    else:
        market_cell = None
    return market_cell



import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.externals import joblib

PREFIX_PATH = 'scoring/models/'
column_order = pd.read_table(PREFIX_PATH+"final_ts.fmap", header=None)[1].tolist()
feature_names =  ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f161', 'f162', 'f163', 'f164', 'f165', 'f166', 'f167', 'f168', 'f169', 'f170', 'f171', 'f172', 'f173', 'f174', 'f175', 'f176', 'f177', 'f178', 'f179', 'f180', 'f181', 'f182', 'f183', 'f184', 'f185', 'f186', 'f187', 'f188', 'f189', 'f190', 'f191', 'f192', 'f193', 'f194', 'f195', 'f196', 'f197', 'f198', 'f199', 'f200', 'f201', 'f202', 'f203', 'f204', 'f205', 'f206', 'f207', 'f208', 'f209', 'f210', 'f211', 'f212', 'f213', 'f214', 'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222', 'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230', 'f231', 'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238', 'f239', 'f240', 'f241', 'f242', 'f243', 'f244', 'f245', 'f246', 'f247', 'f248', 'f249', 'f250', 'f251', 'f252', 'f253', 'f254', 'f255', 'f256', 'f257', 'f258', 'f259', 'f260', 'f261', 'f262', 'f263', 'f264', 'f265', 'f266', 'f267', 'f268', 'f269', 'f270', 'f271', 'f272', 'f273', 'f274', 'f275', 'f276', 'f277', 'f278', 'f279', 'f280', 'f281', 'f282', 'f283', 'f284', 'f285', 'f286', 'f287', 'f288', 'f289', 'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296', 'f297', 'f298', 'f299', 'f300', 'f301', 'f302', 'f303', 'f304', 'f305', 'f306', 'f307', 'f308', 'f309', 'f310', 'f311', 'f312', 'f313', 'f314', 'f315', 'f316', 'f317', 'f318', 'f319', 'f320', 'f321', 'f322', 'f323', 'f324', 'f325', 'f326', 'f327', 'f328', 'f329', 'f330', 'f331', 'f332', 'f333', 'f334', 'f335', 'f336', 'f337', 'f338', 'f339', 'f340', 'f341', 'f342', 'f343', 'f344', 'f345', 'f346', 'f347', 'f348', 'f349', 'f350', 'f351', 'f352', 'f353', 'f354', 'f355', 'f356', 'f357', 'f358', 'f359', 'f360', 'f361', 'f362', 'f363', 'f364', 'f365', 'f366', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'f373', 'f374', 'f375', 'f376', 'f377', 'f378', 'f379', 'f380', 'f381', 'f382', 'f383', 'f384', 'f385', 'f386', 'f387', 'f388', 'f389', 'f390', 'f391', 'f392', 'f393', 'f394', 'f395', 'f396', 'f397', 'f398', 'f399', 'f400', 'f401', 'f402', 'f403', 'f404', 'f405', 'f406', 'f407', 'f408', 'f409', 'f410', 'f411', 'f412', 'f413', 'f414', 'f415', 'f416', 'f417', 'f418', 'f419', 'f420', 'f421', 'f422', 'f423', 'f424', 'f425', 'f426', 'f427', 'f428', 'f429', 'f430', 'f431', 'f432', 'f433', 'f434', 'f435', 'f436', 'f437', 'f438', 'f439', 'f440', 'f441', 'f442', 'f443', 'f444', 'f445', 'f446', 'f447', 'f448', 'f449', 'f450', 'f451', 'f452', 'f453', 'f454', 'f455', 'f456', 'f457', 'f458', 'f459', 'f460', 'f461', 'f462', 'f463', 'f464', 'f465', 'f466', 'f467', 'f468', 'f469', 'f470', 'f471', 'f472', 'f473', 'f474', 'f475', 'f476', 'f477', 'f478', 'f479', 'f480', 'f481', 'f482', 'f483', 'f484', 'f485', 'f486', 'f487', 'f488', 'f489', 'f490', 'f491', 'f492', 'f493', 'f494', 'f495', 'f496', 'f497', 'f498', 'f499', 'f500', 'f501', 'f502', 'f503', 'f504', 'f505', 'f506', 'f507', 'f508', 'f509', 'f510', 'f511', 'f512', 'f513', 'f514', 'f515', 'f516', 'f517', 'f518', 'f519', 'f520', 'f521', 'f522', 'f523', 'f524', 'f525', 'f526', 'f527', 'f528', 'f529', 'f530', 'f531', 'f532', 'f533', 'f534', 'f535', 'f536', 'f537', 'f538', 'f539', 'f540', 'f541', 'f542', 'f543', 'f544', 'f545', 'f546', 'f547', 'f548', 'f549', 'f550', 'f551', 'f552', 'f553', 'f554', 'f555', 'f556', 'f557', 'f558', 'f559', 'f560', 'f561', 'f562', 'f563', 'f564', 'f565', 'f566', 'f567', 'f568', 'f569', 'f570', 'f571', 'f572', 'f573', 'f574', 'f575', 'f576', 'f577', 'f578', 'f579', 'f580', 'f581', 'f582', 'f583', 'f584', 'f585', 'f586', 'f587', 'f588', 'f589', 'f590', 'f591', 'f592', 'f593', 'f594', 'f595', 'f596', 'f597', 'f598', 'f599', 'f600', 'f601', 'f602', 'f603', 'f604', 'f605', 'f606', 'f607', 'f608', 'f609', 'f610', 'f611', 'f612', 'f613', 'f614', 'f615', 'f616', 'f617', 'f618', 'f619', 'f620', 'f621', 'f622', 'f623', 'f624', 'f625', 'f626', 'f627', 'f628', 'f629', 'f630', 'f631', 'f632', 'f633', 'f634', 'f635', 'f636', 'f637', 'f638', 'f639', 'f640', 'f641', 'f642', 'f643', 'f644', 'f645', 'f646', 'f647', 'f648', 'f649', 'f650', 'f651', 'f652', 'f653', 'f654', 'f655', 'f656', 'f657', 'f658', 'f659', 'f660', 'f661', 'f662', 'f663', 'f664', 'f665', 'f666', 'f667', 'f668', 'f669', 'f670', 'f671', 'f672', 'f673', 'f674', 'f675', 'f676', 'f677', 'f678', 'f679', 'f680', 'f681', 'f682', 'f683', 'f684', 'f685', 'f686', 'f687', 'f688', 'f689', 'f690', 'f691', 'f692', 'f693', 'f694', 'f695', 'f696', 'f697', 'f698', 'f699', 'f700', 'f701', 'f702', 'f703', 'f704', 'f705', 'f706', 'f707', 'f708', 'f709', 'f710', 'f711', 'f712', 'f713', 'f714', 'f715', 'f716', 'f717', 'f718', 'f719', 'f720', 'f721', 'f722', 'f723', 'f724', 'f725', 'f726', 'f727', 'f728', 'f729', 'f730', 'f731', 'f732', 'f733', 'f734', 'f735', 'f736', 'f737', 'f738', 'f739', 'f740', 'f741', 'f742', 'f743', 'f744', 'f745', 'f746', 'f747', 'f748', 'f749', 'f750', 'f751', 'f752', 'f753', 'f754', 'f755', 'f756', 'f757', 'f758', 'f759', 'f760', 'f761', 'f762', 'f763', 'f764', 'f765', 'f766', 'f767', 'f768', 'f769', 'f770', 'f771', 'f772', 'f773', 'f774', 'f775', 'f776', 'f777', 'f778', 'f779', 'f780', 'f781', 'f782', 'f783', 'f784', 'f785', 'f786', 'f787', 'f788', 'f789', 'f790', 'f791', 'f792', 'f793', 'f794', 'f795', 'f796', 'f797', 'f798', 'f799', 'f800', 'f801', 'f802', 'f803', 'f804', 'f805', 'f806', 'f807', 'f808', 'f809', 'f810', 'f811', 'f812', 'f813', 'f814', 'f815', 'f816', 'f817', 'f818', 'f819', 'f820', 'f821', 'f822', 'f823', 'f824', 'f825', 'f826', 'f827', 'f828', 'f829', 'f830', 'f831', 'f832', 'f833', 'f834', 'f835', 'f836', 'f837', 'f838', 'f839', 'f840', 'f841', 'f842', 'f843', 'f844', 'f845', 'f846', 'f847', 'f848', 'f849', 'f850', 'f851', 'f852', 'f853', 'f854', 'f855', 'f856', 'f857', 'f858', 'f859', 'f860', 'f861', 'f862', 'f863', 'f864', 'f865', 'f866', 'f867', 'f868', 'f869', 'f870', 'f871', 'f872', 'f873', 'f874', 'f875', 'f876', 'f877', 'f878', 'f879', 'f880', 'f881', 'f882', 'f883', 'f884', 'f885', 'f886', 'f887', 'f888', 'f889', 'f890', 'f891', 'f892', 'f893', 'f894', 'f895', 'f896', 'f897', 'f898', 'f899', 'f900', 'f901', 'f902', 'f903', 'f904', 'f905', 'f906', 'f907', 'f908', 'f909', 'f910', 'f911', 'f912', 'f913', 'f914', 'f915', 'f916', 'f917', 'f918', 'f919', 'f920', 'f921', 'f922', 'f923', 'f924', 'f925', 'f926', 'f927', 'f928', 'f929', 'f930', 'f931', 'f932', 'f933', 'f934', 'f935', 'f936', 'f937', 'f938', 'f939', 'f940', 'f941', 'f942', 'f943', 'f944', 'f945', 'f946', 'f947', 'f948', 'f949', 'f950', 'f951', 'f952', 'f953', 'f954', 'f955', 'f956', 'f957', 'f958', 'f959', 'f960', 'f961', 'f962', 'f963', 'f964', 'f965', 'f966', 'f967', 'f968', 'f969', 'f970', 'f971', 'f972', 'f973', 'f974', 'f975', 'f976', 'f977', 'f978', 'f979', 'f980', 'f981', 'f982', 'f983', 'f984', 'f985', 'f986', 'f987', 'f988', 'f989', 'f990', 'f991', 'f992', 'f993', 'f994', 'f995', 'f996', 'f997', 'f998', 'f999', 'f1000', 'f1001', 'f1002', 'f1003', 'f1004', 'f1005', 'f1006', 'f1007', 'f1008', 'f1009', 'f1010', 'f1011', 'f1012', 'f1013']

def set_trio_flag(df):
    df = df.copy()
    df['truly_simple_flag'] = 0
    return df

def set_ts_flag(df):
    df = df.copy()
    df['truly_simple_flag'] = 1
    return df

def score_trio_model(df):
    loaded_model = joblib.load(PREFIX_PATH+'final_trio_xgboost.mod')
    trio_df =df.copy().pipe(set_trio_flag).pipe(lambda x: x[column_order])
    scoring_data = xgb.DMatrix(trio_df[column_order],feature_names=feature_names)

    return loaded_model.predict(scoring_data)

def score_ts_model(df):
    loaded_model = joblib.load(PREFIX_PATH+'final_ts_xgboost.mod')
    ts_df =df.copy().pipe(set_ts_flag).pipe(lambda x: x[column_order])
    scoring_data = xgb.DMatrix(ts_df[column_order],feature_names=feature_names)

    return loaded_model.predict(scoring_data)

def score(full_file_df):
    df = full_file_df.copy()
    df['scored_trio_uncalibrated'] = np.round(score_trio_model(df),6)
    df['scored_ts_uncalibrated']=np.round(score_ts_model(df),6)
    out_columns =['TU_Edit_Sequence_Number','BC31', 'AT35', 'BC21', 'CEMP09', 'scored_trio_uncalibrated','scored_ts_uncalibrated']
    return df[out_columns]






import numpy as np
import dask.dataframe as dd
import pandas as pd
from numpy import dtype
meta_dtypes =  {'PB24S': dtype('int64'), 'LM21S': dtype('int64'), 'IN29S': dtype('int64'), 'AT14': dtype('int64'), 'G224A': dtype('int64'), 'MT28S': dtype('int64'), 'Existing_Customer_Tag': dtype('float64'), 'S061S': dtype('int64'), 'PB33': dtype('int64'), 'G057S': dtype('int64'), 'G106S': dtype('int64'), 'BKC323': dtype('int64'), 'S115': dtype('int64'), 'AT01': dtype('int64'), 'EADS51_Score': dtype('int64'), 'S018': dtype('int64'), 'S059': dtype('int64'), 'RE34': dtype('float64'), 'G307S': dtype('int64'), 'BI06S': dtype('int64'), 'PF02': dtype('int64'), 'MD02S': dtype('int64'), 'BC21S': dtype('int64'), 'LM13S': dtype('int64'), 'BC107S': dtype('int64'), 'RT07': dtype('int64'), 'AT25': dtype('int64'), 'CO03S': dtype('int64'), 'S063S': dtype('int64'), 'CV16': dtype('int64'), 'MT04': dtype('int64'), 'BKC315': dtype('int64'), 'BR29S': dtype('int64'), 'BC05': dtype('int64'), 'LM09S': dtype('int64'), 'CV22': dtype('int64'), 'OF03': dtype('int64'), 'BC57S': dtype('int64'), 'PNC_Spend_Score_ARGUS': dtype('float64'), 'BKC152': dtype('int64'), 'G220C': dtype('int64'), 'Post_Directional': dtype('float64'), 'AT27': dtype('int64'), 'AT36': dtype('float64'), 'PB05': dtype('int64'), 'MT27S': dtype('int64'), 'G201A': dtype('int64'), 'LM29S': dtype('int64'), 'PB08': dtype('int64'), 'CV23': dtype('int64'), 'S065': dtype('int64'), 'UR14': dtype('int64'), 'BR27S': dtype('int64'), 'S071B': dtype('int64'), 'Street_Suffix': dtype('float64'), 'S073A': dtype('int64'), 'MT30S': dtype('int64'), 'RT03': dtype('int64'), 'FI27S': dtype('int64'), 'INAP01': dtype('int64'), 'FI30S': dtype('int64'), 'FI101S': dtype('int64'), 'G312S': dtype('int64'), 'BKC12': dtype('int64'), 'FI02S': dtype('int64'), 'HR35S': dtype('int64'), 'G028': dtype('int64'), 'RT33S': dtype('int64'), 'G030': dtype('int64'), 'G240S': dtype('int64'), 'BR33S': dtype('int64'), 'HI03S': dtype('int64'), 'RP02S': dtype('int64'), 'FI05': dtype('int64'), 'thin_flag': dtype('int64'), 'New': dtype('float64'), 'AGG604': dtype('int64'), 'MT33S': dtype('int64'), 'PB13': dtype('int64'), 'G016': dtype('int64'), 'First_Name': dtype('float64'), 'PB09S': dtype('int64'), 'RE27S': dtype('int64'), 'HR24S': dtype('int64'), 'HI02S': dtype('int64'), 'RP06S': dtype('int64'), 'LM37S': dtype('int64'), 'G221A': dtype('int64'), 'OF12S': dtype('int64'), 'UR28': dtype('int64'), 'PB06': dtype('int64'), 'ST103S': dtype('int64'), 'IN28S': dtype('int64'), 'CO06S': dtype('int64'), 'MT06S': dtype('int64'), 'G066': dtype('int64'), 'PF06': dtype('int64'), 'G243F': dtype('int64'), 'RT36S': dtype('int64'), 'G088': dtype('int64'), 'Affiliate': dtype('float64'), 'RT08': dtype('int64'), 'S004': dtype('int64'), 'MT36': dtype('float64'), 'G301S': dtype('int64'), 'G221D': dtype('int64'), 'RE14': dtype('int64'), 'FR34S': dtype('int64'), 'BC28S': dtype('int64'), 'G304S': dtype('int64'), 'RE35S': dtype('int64'), 'AGG613': dtype('int64'), 'RT28': dtype('int64'), 'S078': dtype('float64'), 'S008': dtype('int64'), 'AGG104': dtype('int64'), 'HI01S': dtype('int64'), 'BC98A': dtype('int64'), 'FC06S': dtype('int64'), 'IN10': dtype('int64'), 'LM23S': dtype('int64'), 'BI35S': dtype('int64'), 'G231S': dtype('int64'), 'MT50': dtype('int64'), 'AT34B': dtype('int64'), 'BC12': dtype('int64'), 'G043': dtype('int64'), 'AT09S': dtype('int64'), 'AT101B': dtype('int64'), 'BC33S': dtype('int64'), 'S068A': dtype('int64'), 'FI09': dtype('int64'), 'Phone_Number': dtype('float64'), 'FC01S': dtype('int64'), 'BC29': dtype('int64'), 'RT03S': dtype('int64'), 'AT35A': dtype('int64'), 'BR28': dtype('int64'), 'HR28S': dtype('int64'), 'G226S': dtype('int64'), 'BR21S': dtype('int64'), 'FICO_Risk_Score_Classic_2004': dtype('float64'), 'LM26S': dtype('int64'), 'BKC11': dtype('int64'), 'G221B': dtype('int64'), 'LM38S': dtype('int64'), 'AU31S': dtype('int64'), 'G002B': dtype('int64'), 'ST28S': dtype('int64'), 'G244F': dtype('int64'), 'FI03': dtype('int64'), 'OF27S': dtype('int64'), 'MT53': dtype('int64'), 'PF08': dtype('int64'), 'S063A': dtype('int64'), 'BKC102': dtype('int64'), 'RT33': dtype('int64'), 'PB32S': dtype('int64'), 'AU101S': dtype('int64'), 'OF35S': dtype('int64'), 'AT02S': dtype('int64'), 'G043S': dtype('int64'), 'AT21S': dtype('int64'), 'FR33': dtype('int64'), 'MT36S': dtype('int64'), 'RT201S': dtype('int64'), 'MT24S': dtype('int64'), 'G980S': dtype('int64'), 'S079': dtype('float64'), 'RT29': dtype('int64'), 'S204A': dtype('int64'), 'BR33': dtype('int64'), 'BR06S': dtype('int64'), 'RT21': dtype('float64'), 'BC106S': dtype('int64'), 'S061': dtype('float64'), 'S071A': dtype('int64'), 'SSN': dtype('float64'), 'BKC322': dtype('int64'), 'MT20': dtype('float64'), 'ON03': dtype('int64'), 'G064S': dtype('int64'), 'AT28B': dtype('int64'), 'G099S': dtype('int64'), 'G057': dtype('int64'), 'AT28A': dtype('int64'), 'MT101S': dtype('int64'), 'MT54': dtype('int64'), 'S054': dtype('int64'), 'UR08': dtype('int64'), 'G001S': dtype('int64'), 'G083': dtype('int64'), 'HI32S': dtype('int64'), 'BKC325': dtype('int64'), 'ST102S': dtype('int64'), 'FR02S': dtype('int64'), 'BR28S': dtype('int64'), 'G062': dtype('int64'), 'LM05S': dtype('int64'), 'BI36S': dtype('int64'), 'BAUT2': dtype('int64'), 'G004': dtype('int64'), 'G251A': dtype('int64'), 'FR35S': dtype('int64'), 'IN36S': dtype('int64'), 'MT25S': dtype('int64'), 'G207S': dtype('int64'), 'OF33S': dtype('int64'), 'S043': dtype('int64'), 'Revenue_Score_Deciles': dtype('int64'), 'BI20': dtype('float64'), 'IN11': dtype('int64'), 'G064': dtype('int64'), 'AT26': dtype('int64'), 'OF30S': dtype('int64'), 'Fifth_Third_Bk_Trade_Tag': dtype('float64'), 'BI01': dtype('int64'), 'DS14': dtype('int64'), 'Cell_Tag': dtype('float64'), 'G221C': dtype('int64'), 'G218A': dtype('int64'), 'BC34S': dtype('int64'), 'G215B': dtype('int64'), 'OF29': dtype('int64'), 'PB06S': dtype('int64'), 'LM17S': dtype('int64'), 'RT29S': dtype('int64'), 'G061': dtype('int64'), 'UR10': dtype('int64'), 'BKC316': dtype('int64'), 'PF33': dtype('int64'), 'OF31S': dtype('int64'), 'AGG213': dtype('int64'), 'BR09S': dtype('int64'), 'MT21': dtype('float64'), 'RE21S': dtype('int64'), 'RE13': dtype('int64'), 'AT29S': dtype('int64'), 'S114': dtype('int64'), 'G235S': dtype('int64'), 'Response_Score_Deciles': dtype('int64'), 'G302S': dtype('int64'), 'G250C': dtype('int64'), 'G063': dtype('int64'), 'OF36': dtype('float64'), 'DS21': dtype('float64'), 'LM31S': dtype('int64'), 'Filler5': dtype('float64'), 'HI25S': dtype('int64'), 'BI25S': dtype('int64'), 'FI09S': dtype('int64'), 'IN21S': dtype('int64'), 'AT10': dtype('int64'), 'BKC324': dtype('int64'), 'G202A': dtype('int64'), 'G244B': dtype('int64'), 'IN24S': dtype('int64'), 'BI09': dtype('int64'), 'Filler4': dtype('float64'), 'CO04S': dtype('int64'), 'DS07': dtype('int64'), 'MT34B': dtype('float64'), 'FR09S': dtype('int64'), 'ON34': dtype('float64'), 'S027': dtype('float64'), 'S066': dtype('int64'), 'PB09': dtype('int64'), 'MT56': dtype('int64'), 'Street_Name': dtype('float64'), 'BKC82': dtype('int64'), 'Promotion_Name': dtype('float64'), 'G242S': dtype('int64'), 'FR29S': dtype('int64'), 'FI03S': dtype('int64'), 'RT05': dtype('int64'), 'BC31': dtype('float64'), 'MT52': dtype('int64'), 'G206S': dtype('int64'), 'BKC122': dtype('int64'), 'S055': dtype('int64'), 'AT35': dtype('float64'), 'BAUT5': dtype('int64'), 'FI36S': dtype('int64'), 'OF29S': dtype('int64'), 'PNC_Behaviour_Segment_ARGUS': dtype('float64'), 'IN03S': dtype('int64'), 'AT34': dtype('float64'), 'BC104S': dtype('int64'), 'IN02S': dtype('int64'), 'MT28': dtype('int64'), 'IN07': dtype('int64'), 'Direction': dtype('float64'), 'HR101S': dtype('int64'), 'G047': dtype('int64'), 'S114S': dtype('int64'), 'AU28S': dtype('int64'), 'G305S': dtype('int64'), 'FI57S': dtype('int64'), 'AT34A': dtype('int64'), 'G250A': dtype('int64'), 'AGG911': dtype('float64'), 'G104S': dtype('int64'), 'BI07': dtype('int64'), 'HI28S': dtype('int64'), 'TU_Edit_Sequence_Number': dtype('int64'), 'RT21S': dtype('int64'), 'G087': dtype('int64'), 'G213B': dtype('int64'), 'G093': dtype('int64'), 'UR13': dtype('int64'), 'G085': dtype('int64'), 'RE02S': dtype('int64'), 'S064B': dtype('int64'), 'HR30S': dtype('int64'), 'BC10': dtype('int64'), 'PF34': dtype('float64'), 'AT07': dtype('int64'), 'CO02S': dtype('int64'), 'AU21S': dtype('int64'), 'AT28': dtype('int64'), 'MD33S': dtype('int64'), 'City': dtype('float64'), 'BI57S': dtype('int64'), 'G224B': dtype('int64'), 'IN30S': dtype('int64'), 'G100S': dtype('int64'), 'G069S': dtype('int64'), 'HR29S': dtype('int64'), 'G086': dtype('int64'), 'AT09': dtype('int64'), 'Non_Parsed_Address': dtype('float64'), 'G022': dtype('int64'), 'BC101S': dtype('int64'), 'RE12': dtype('int64'), 'AT33B': dtype('int64'), 'BC02S': dtype('int64'), 'HI33S': dtype('int64'), 'OF28': dtype('int64'), 'RP01S': dtype('int64'), 'G094': dtype('int64'), 'S0Y2': dtype('float64'), 'BC09S': dtype('int64'), 'BC35S': dtype('int64'), 'G222S': dtype('int64'), 'G215A': dtype('int64'), 'G243B': dtype('int64'), 'RE32S': dtype('int64'), 'AT25S': dtype('int64'), 'MT01S': dtype('int64'), 'G002S': dtype('int64'), 'RT101S': dtype('int64'), 'BC13': dtype('int64'), 'BKC83': dtype('int64'), 'FR21S': dtype('int64'), 'G225S': dtype('int64'), 'RP04S': dtype('int64'), 'G084': dtype('int64'), 'RE12S': dtype('int64'), 'IN13': dtype('int64'), 'AU01S': dtype('int64'), 'BI101S': dtype('int64'), 'G058': dtype('int64'), 'HI34S': dtype('int64'), 'ST31S': dtype('int64'), 'HIAP01': dtype('int64'), 'HR21S': dtype('int64'), 'G007': dtype('int64'), 'AGG607': dtype('int64'), 'BI06': dtype('int64'), 'AT12S': dtype('int64'), 'MT57': dtype('int64'), 'S068B': dtype('int64'), 'IN57S': dtype('int64'), 'AGG501': dtype('int64'), 'DS06': dtype('int64'), 'Run_Date': dtype('int64'), 'G219A': dtype('int64'), 'RE31S': dtype('int64'), 'AU25S': dtype('int64'), 'RE35': dtype('float64'), 'RE09S': dtype('int64'), 'OF34S': dtype('int64'), 'G960S': dtype('int64'), 'HRAP01': dtype('int64'), 'G219D': dtype('int64'), 'LM08S': dtype('int64'), 'MT35S': dtype('int64'), 'FI25S': dtype('int64'), 'BI03': dtype('int64'), 'OF57S': dtype('int64'), 'G096': dtype('int64'), 'MT29': dtype('int64'), 'ST50S': dtype('int64'), 'ST01S': dtype('int64'), 'MT41': dtype('int64'), 'CV17': dtype('int64'), 'OF06S': dtype('int64'), 'FR01S': dtype('int64'), 'ST02S': dtype('int64'), 'S060': dtype('int64'), 'RP05S': dtype('int64'), 'G027': dtype('int64'), 'BKC84': dtype('int64'), 'BI21S': dtype('int64'), 'DS05': dtype('int64'), 'G005': dtype('int64'), 'S015': dtype('int64'), 'G044': dtype('int64'), 'BI31S': dtype('int64'), 'G019': dtype('int64'), 'G094S': dtype('int64'), 'CV30': dtype('int64'), 'S010': dtype('int64'), 'BC11': dtype('int64'), 'RT34': dtype('float64'), 'RP03S': dtype('int64'), 'Filler13': dtype('float64'), 'RE24S': dtype('int64'), 'IN35S': dtype('int64'), 'IN34': dtype('float64'), 'BI09S': dtype('int64'), 'G313S': dtype('int64'), 'G201B': dtype('int64'), 'AU24S': dtype('int64'), 'G048': dtype('int64'), 'BC02': dtype('int64'), 'MT34': dtype('float64'), 'FC04S': dtype('int64'), 'AT01A': dtype('int64'), 'IN20S': dtype('int64'), 'G212S': dtype('int64'), 'MT03S': dtype('int64'), 'BKC318': dtype('int64'), 'G242B': dtype('int64'), 'G233S': dtype('int64'), 'CV26': dtype('int64'), 'HI57S': dtype('int64'), 'FTB16': dtype('int64'), 'FR57S': dtype('int64'), 'CV24': dtype('int64'), 'ST45S': dtype('int64'), 'G303S': dtype('int64'), 'AU30S': dtype('int64'), 'AT08': dtype('int64'), 'S009': dtype('int64'), 'REAP01': dtype('int64'), 'PB20S': dtype('int64'), 'Filler14': dtype('float64'), 'RT09': dtype('int64'), 'HR33S': dtype('int64'), 'BC24S': dtype('int64'), 'AT27S': dtype('int64'), 'BR32S': dtype('int64'), 'LM30S': dtype('int64'), 'HI36S': dtype('int64'), 'PB10': dtype('int64'), 'PB36S': dtype('int64'), 'BC21': dtype('float64'), 'G018': dtype('int64'), 'G242F': dtype('int64'), 'BKC13': dtype('int64'), 'BI20S': dtype('int64'), 'G065': dtype('int64'), 'MT01': dtype('int64'), 'PB31S': dtype('int64'), 'G095S': dtype('int64'), 'RP08S': dtype('int64'), 'LM24S': dtype('int64'), 'IN33S': dtype('int64'), 'AGG907': dtype('int64'), 'S019': dtype('int64'), 'OF21S': dtype('int64'), 'AT104S': dtype('int64'), 'BR30S': dtype('int64'), 'RT01S': dtype('int64'), 'CO01S': dtype('int64'), 'HR25S': dtype('int64'), 'LM25S': dtype('int64'), 'S206S': dtype('int64'), 'BI27S': dtype('int64'), 'RE33S': dtype('int64'), 'G068S': dtype('int64'), 'AT23': dtype('int64'), 'G227S': dtype('int64'), 'FC05S': dtype('int64'), 'IN32S': dtype('int64'), 'G042S': dtype('int64'), 'G219B': dtype('int64'), 'Filler6': dtype('float64'), 'G990S': dtype('int64'), 'BC06S': dtype('int64'), 'AU33S': dtype('int64'), 'G021': dtype('int64'), 'FR24S': dtype('int64'), 'LM07S': dtype('int64'), 'G309S': dtype('int64'), 'G059': dtype('int64'), 'MT46': dtype('int64'), 'PB29S': dtype('int64'), 'ST99S': dtype('int64'), 'G243S': dtype('int64'), 'MT12S': dtype('int64'), 'S020': dtype('int64'), 'IN31S': dtype('int64'), 'G063S': dtype('int64'), 'MTAP01': dtype('int64'), 'ON20': dtype('float64'), 'OF03S': dtype('int64'), 'PB35': dtype('float64'), 'RE32': dtype('int64'), 'HR34S': dtype('int64'), 'AU06S': dtype('int64'), 'BKC14': dtype('int64'), 'ST06S': dtype('int64'), 'AGG101': dtype('int64'), 'AT06': dtype('int64'), 'CV18': dtype('int64'), 'ON01': dtype('int64'), 'RT24S': dtype('int64'), 'AU20S': dtype('int64'), 'G232S': dtype('int64'), 'CV02': dtype('int64'), 'RT35': dtype('float64'), 'HR03S': dtype('int64'), 'Filler10': dtype('float64'), 'HR12S': dtype('int64'), 'G224C': dtype('int64'), 'CV15': dtype('int64'), 'G099A': dtype('int64'), 'BKC132': dtype('int64'), 'AU12S': dtype('int64'), 'BC07': dtype('int64'), 'BC97A': dtype('int64'), 'G230S': dtype('int64'), 'UR05': dtype('int64'), 'PF09': dtype('int64'), 'ST29S': dtype('int64'), 'ST05S': dtype('int64'), 'AGG201': dtype('int64'), 'RE20': dtype('float64'), 'ST20S': dtype('int64'), 'HR01S': dtype('int64'), 'RT13': dtype('int64'), 'LM01S': dtype('int64'), 'BC29S': dtype('int64'), 'FI08': dtype('int64'), 'BC06': dtype('int64'), 'DS33': dtype('int64'), 'Vantage_Score_3': dtype('int64'), 'G218B': dtype('int64'), 'FI34S': dtype('int64'), 'UR11': dtype('int64'), 'RE36S': dtype('int64'), 'RE57S': dtype('int64'), 'LM39S': dtype('int64'), 'G068': dtype('int64'), 'AT20S': dtype('int64'), 'G238S': dtype('int64'), 'MT33': dtype('int64'), 'OF20': dtype('float64'), 'RE20S': dtype('int64'), 'House_Number': dtype('float64'), 'IN27S': dtype('int64'), 'PB27S': dtype('int64'), 'MT22': dtype('float64'), 'G041C': dtype('int64'), 'RT35S': dtype('int64'), 'FI24S': dtype('int64'), 'G104': dtype('int64'), 'BKC319': dtype('int64'), 'G003C': dtype('int64'), 'BR20S': dtype('int64'), 'PF03': dtype('int64'), 'G217S': dtype('int64'), 'BR24S': dtype('int64'), 'RT11': dtype('int64'), 'DS10': dtype('int64'), 'CV05': dtype('int64'), 'Last_Name': dtype('float64'), 'AU51A': dtype('int64'), 'Marketing_Code': dtype('float64'), 'AGG204': dtype('int64'), 'G239S': dtype('int64'), 'G093S': dtype('int64'), 'State': dtype('O'), 'Urbanization': dtype('float64'), 'BC03S': dtype('int64'), 'RE01S': dtype('int64'), 'AT12': dtype('int64'), 'AT101S': dtype('int64'), 'IN09S': dtype('int64'), 'G023': dtype('int64'), 'IN101S': dtype('int64'), 'FR33S': dtype('int64'), 'CV14': dtype('int64'), 'RE33': dtype('int64'), 'BKC317': dtype('int64'), 'S204S': dtype('int64'), 'PB57S': dtype('int64'), 'DS09': dtype('int64'), 'IN03': dtype('int64'), 'HR57S': dtype('int64'), 'HI06S': dtype('int64'), 'AT35B': dtype('int64'), 'BC27S': dtype('int64'), 'RT30S': dtype('int64'), 'Source_Code': dtype('float64'), 'AU27S': dtype('int64'), 'FI33S': dtype('int64'), 'G043C': dtype('int64'), 'G089': dtype('float64'), 'RT10': dtype('int64'), 'DS12': dtype('int64'), 'BKC112': dtype('int64'), 'G251B': dtype('int64'), 'OF02S': dtype('int64'), 'RT01': dtype('int64'), 'G026': dtype('int64'), 'OF28S': dtype('int64'), 'Filler1': dtype('float64'), 'BI28': dtype('int64'), 'FR27S': dtype('int64'), 'MT03': dtype('int64'), 'MT47': dtype('int64'), 'G059S': dtype('int64'), 'AT20': dtype('int64'), 'MD106S': dtype('int64'), 'DS02': dtype('int64'), 'AT03': dtype('int64'), 'BKC327': dtype('int64'), 'OF101S': dtype('int64'), 'FR30S': dtype('int64'), 'IN08': dtype('int64'), 'MT02S': dtype('int64'), 'RE11': dtype('int64'), 'FI01': dtype('int64'), 'BC03': dtype('int64'), 'AGG207': dtype('int64'), 'G008': dtype('int64'), 'FR20S': dtype('int64'), 'DS08': dtype('int64'), 'HR32S': dtype('int64'), 'G200S': dtype('int64'), 'PB07': dtype('int64'), 'Carrier_Route': dtype('float64'), 'MT43': dtype('int64'), 'S207S': dtype('int64'), 'LM04S': dtype('int64'), 'CV04': dtype('int64'), 'OF36S': dtype('int64'), 'MT47S': dtype('int64'), 'UR35': dtype('float64'), 'S209A': dtype('int64'), 'HI24S': dtype('int64'), 'FR28S': dtype('int64'), 'MT32': dtype('int64'), 'TRB01TOT': dtype('int64'), 'Filler7': dtype('float64'), 'S063': dtype('int64'), 'UR03': dtype('int64'), 'AU02S': dtype('int64'), 'DS13': dtype('int64'), 'G095': dtype('float64'), 'OF09S': dtype('int64'), 'FI28S': dtype('int64'), 'CV19': dtype('int64'), 'HI20S': dtype('int64'), 'G071S': dtype('int64'), 'LM36S': dtype('int64'), 'RT34S': dtype('int64'), 'G250B': dtype('int64'), 'G003': dtype('int64'), 'G244S': dtype('int64'), 'PB03S': dtype('int64'), 'G098': dtype('int64'), 'G102': dtype('float64'), 'IN21': dtype('float64'), 'RT27S': dtype('int64'), 'G308S': dtype('int64'), 'BR31S': dtype('int64'), 'BI33S': dtype('int64'), 'OF32S': dtype('int64'), 'CV11': dtype('int64'), 'FR03': dtype('int64'), 'Product_Code': dtype('float64'), 'BI24S': dtype('int64'), 'AU09S': dtype('int64'), 'ST21S': dtype('int64'), 'RT12S': dtype('int64'), 'G017': dtype('int64'), 'IN33': dtype('int64'), 'G103': dtype('int64'), 'BC102S': dtype('int64'), 'PB21': dtype('float64'), 'BC36S': dtype('int64'), 'BC98': dtype('float64'), 'FI12S': dtype('int64'), 'RT12': dtype('int64'), 'FI06': dtype('int64'), 'OF01': dtype('int64'), 'CV21': dtype('int64'), 'IN05': dtype('int64'), 'G060': dtype('int64'), 'LM02S': dtype('int64'), 'BKC320': dtype('int64'), 'G067': dtype('int64'), 'RE25S': dtype('int64'), 'RT57S': dtype('int64'), 'S004S': dtype('int64'), 'UR20': dtype('float64'), 'LM10S': dtype('int64'), 'AGG909': dtype('int64'), 'ST12S': dtype('int64'), 'BKC142': dtype('int64'), 'Filler3': dtype('float64'), 'LM32S': dtype('int64'), 'RE30S': dtype('int64'), 'CV28': dtype('int64'), 'PB01S': dtype('int64'), 'S043S': dtype('int64'), 'FI31S': dtype('int64'), 'UR09': dtype('int64'), 'G218C': dtype('int64'), 'PB12S': dtype('int64'), 'AGG504': dtype('int64'), 'LM14S': dtype('int64'), 'CO07S': dtype('int64'), 'EFTB08_Revenue_Score': dtype('float64'), 'BI03S': dtype('int64'), 'Promo_Code': dtype('float64'), 'Occupation_Code': dtype('float64'), 'IN12': dtype('int64'), 'Apartment': dtype('float64'), 'Filler2': dtype('float64'), 'MT35': dtype('float64'), 'ATAP01': dtype('int64'), 'RE06S': dtype('int64'), 'G311S': dtype('int64'), 'OF20S': dtype('int64'), 'HR06S': dtype('int64'), 'ST35S': dtype('int64'), 'G071': dtype('int64'), 'MT51': dtype('int64'), 'HI27S': dtype('int64'), 'PB34S': dtype('int64'), 'FR25S': dtype('int64'), 'AT99': dtype('int64'), 'RE28S': dtype('int64'), 'BR35S': dtype('int64'), 'BAUT3': dtype('int64'), 'MT55': dtype('float64'), 'OF24S': dtype('int64'), 'S208S': dtype('int64'), 'BKC52': dtype('int64'), 'AT32S': dtype('int64'), 'S040': dtype('int64'), 'ST101S': dtype('int64'), 'FR101S': dtype('int64'), 'RT14': dtype('int64'), 'LM22S': dtype('int64'), 'G223S': dtype('int64'), 'Middle_Name': dtype('float64'), 'AT01S': dtype('int64'), 'G105S': dtype('int64'), 'LM28S': dtype('int64'), 'G049': dtype('int64'), 'RE34S': dtype('int64'), 'FI07': dtype('int64'), 'RT20': dtype('float64'), 'CV09': dtype('int64'), 'G069': dtype('int64'), 'BR12S': dtype('int64'), 'AT103S': dtype('int64'), 'G058S': dtype('int64'), 'Unit_Code': dtype('float64'), 'BC103S': dtype('int64'), 'G020S': dtype('int64'), 'AU35S': dtype('int64'), 'G218D': dtype('int64'), 'BC34': dtype('float64'), 'S064A': dtype('int64'), 'BR101S': dtype('int64'), 'LM35S': dtype('int64'), 'MT09S': dtype('int64'), 'RE10': dtype('int64'), 'AT24': dtype('int64'), 'S046': dtype('float64'), 'RT32S': dtype('int64'), 'IN12S': dtype('int64'), 'RT20S': dtype('int64'), 'Zip': dtype('int64'), 'FI01S': dtype('int64'), 'AGG906': dtype('float64'), 'MT49': dtype('int64'), 'PB02S': dtype('int64'), 'CO05S': dtype('int64'), 'BC08': dtype('int64'), 'AT57S': dtype('int64'), 'BR03S': dtype('int64'), 'Filler11': dtype('float64'), 'HR20S': dtype('int64'), 'HI101S': dtype('int64'), 'MT34C': dtype('float64'), 'LM06S': dtype('int64'), 'AT33': dtype('int64'), 'G042C': dtype('int64'), 'BC35': dtype('float64'), 'G082': dtype('int64'), 'G251C': dtype('int64'), 'G213A': dtype('int64'), 'AT24S': dtype('int64'), 'FR35': dtype('float64'), 'AT33A': dtype('int64'), 'HR36S': dtype('int64'), 'RT31S': dtype('int64'), 'PB28S': dtype('int64'), 'AT06S': dtype('int64'), 'PF05': dtype('int64'), 'AGG507': dtype('int64'), 'HR09S': dtype('int64'), 'Expiration_Date': dtype('float64'), 'LM18S': dtype('int64'), 'DS04': dtype('int64'), 'RT36': dtype('float64'), 'CV27': dtype('int64'), 'MT29S': dtype('int64'), 'RT25S': dtype('int64'), 'IN06': dtype('int64'), 'Filler12': dtype('float64'), 'FI20S': dtype('int64'), 'MT42': dtype('int64'), 'FI06S': dtype('int64'), 'Branch': dtype('float64'), 'G103S': dtype('int64'), 'PB101S': dtype('int64'), 'Filler9': dtype('float64'), 'S073B': dtype('int64'), 'G002': dtype('int64'), 'G241S': dtype('int64'), 'FI35S': dtype('int64'), 'AT31S': dtype('int64'), 'RT28S': dtype('int64'), 'BC109S': dtype('int64'), 'MT31S': dtype('int64'), 'LM16S': dtype('int64'), 'OF33': dtype('int64'), 'EFTB11_Response_Score': dtype('int64'), 'G237S': dtype('int64'), 'PB30S': dtype('int64'), 'CV03': dtype('int64'), 'BKC321': dtype('int64'), 'G020': dtype('int64'), 'UR07': dtype('int64'), 'BKC81': dtype('int64'), 'FR31S': dtype('int64'), 'G202B': dtype('int64'), 'BC12S': dtype('int64'), 'G216S': dtype('int64'), 'G051S': dtype('int64'), 'UR12': dtype('int64'), 'G001C': dtype('int64'), 'CV29': dtype('int64'), 'RT06S': dtype('int64'), 'S062': dtype('float64'), 'BR34S': dtype('int64'), 'RE28': dtype('int64'), 'LM11S': dtype('int64'), 'ST27S': dtype('int64'), 'AGG601': dtype('int64'), 'AT05': dtype('int64'), 'PB11': dtype('int64'), 'G041': dtype('int64'), 'G219C': dtype('int64'), 'G205S': dtype('int64'), 'RT06': dtype('int64'), 'S002': dtype('float64'), 'BKC162': dtype('int64'), 'DS03': dtype('int64'), 'S012': dtype('int64'), 'BI28S': dtype('int64'), 'G002C': dtype('int64'), 'RT09S': dtype('int64'), 'Permid': dtype('float64'), 'G041S': dtype('int64'), 'G310S': dtype('int64'), 'AGG513': dtype('int64'), 'BC36': dtype('float64'), 'G050': dtype('int64'), 'BR36S': dtype('int64'), 'AT13': dtype('int64'), 'LM27S': dtype('int64'), 'ST33S': dtype('int64'), 'LM34S': dtype('int64'), 'AT30S': dtype('int64'), 'MT34S': dtype('int64'), 'RP07S': dtype('int64'), 'BKC54': dtype('int64'), 'LM33S': dtype('int64'), 'BKC326': dtype('int64'), 'IN14': dtype('int64'), 'CV20': dtype('int64'), 'BC30S': dtype('int64'), 'G220D': dtype('int64'), 'G046': dtype('int64'), 'RE03': dtype('int64'), 'CEMP09': dtype('int64'), 'ON33': dtype('int64'), 'FR03S': dtype('int64'), 'G091': dtype('int64'), 'FR12S': dtype('int64'), 'G003S': dtype('int64'), 'RE03S': dtype('int64'), 'ST36S': dtype('int64'), 'G314S': dtype('int64'), 'G006': dtype('int64'), 'BR25S': dtype('int64'), 'ST03S': dtype('int64'), 'BR109S': dtype('int64'), 'G001B': dtype('int64'), 'RT02S': dtype('int64'), 'FC03S': dtype('int64'), 'AT11': dtype('int64'), 'G001': dtype('int64'), 'AT03S': dtype('int64'), 'ST60S': dtype('int64'), 'AGG113': dtype('int64'), 'LM15S': dtype('int64'), 'FI32S': dtype('int64'), 'G208S': dtype('int64'), 'CV13': dtype('int64'), 'FC02S': dtype('int64'), 'BI02S': dtype('int64'), 'Name_Tag': dtype('float64'), 'G199S': dtype('int64'), 'LM03S': dtype('int64'), 'Delivery_Point': dtype('float64'), 'ST32S': dtype('int64'), 'BC25S': dtype('int64'), 'G025': dtype('int64'), 'CV07': dtype('int64'), 'HR27S': dtype('int64'), 'ST24S': dtype('int64'), 'BR03': dtype('int64'), 'G045': dtype('int64'), 'CV10': dtype('int64'), 'ST34S': dtype('int64'), 'BI05': dtype('int64'), 'HR02S': dtype('int64'), 'S011': dtype('int64'), 'BC31S': dtype('int64'), 'FI21S': dtype('int64'), 'PB12': dtype('int64'), 'BI30S': dtype('int64'), 'HR31S': dtype('int64'), 'S014': dtype('int64'), 'LM19S': dtype('int64'), 'ST25S': dtype('int64'), 'BR20': dtype('float64'), 'CV12': dtype('int64'), 'BI32S': dtype('int64'), 'S064': dtype('int64'), 'AGG908': dtype('int64'), 'AT29': dtype('int64'), 'OF25S': dtype('int64'), 'IN06S': dtype('int64'), 'PB25S': dtype('int64'), 'UR06': dtype('int64'), 'AGG910': dtype('float64'), 'G029': dtype('int64'), 'G306S': dtype('int64'), 'FR36S': dtype('int64'), 'G234S': dtype('int64'), 'FR32S': dtype('int64'), 'RE102S': dtype('int64'), 'Criteria_Tag': dtype('int64'), 'BC108S': dtype('int64'), 'AU03S': dtype('int64'), 'G042': dtype('int64'), 'MT44': dtype('int64'), 'G102S': dtype('int64'), 'IN09': dtype('int64'), 'HI31S': dtype('int64'), 'BC01S': dtype('int64'), 'AT21': dtype('int64'), 'BC32S': dtype('int64'), 'Product_Description': dtype('float64'), 'MT02': dtype('int64'), 'FILLERZ': dtype('float64'), 'G009': dtype('int64'), 'BKC92': dtype('int64'), 'PB03': dtype('int64'), 'LM20S': dtype('int64'), 'PF07': dtype('int64'), 'HI09S': dtype('int64'), 'PB33S': dtype('int64'), 'G300S': dtype('int64'), 'PB21S': dtype('int64'), 'ST30S': dtype('int64'), 'AU29S': dtype('int64'), 'MT21S': dtype('int64'), 'ST09S': dtype('int64'), 'S062S': dtype('int64'), 'BKC51': dtype('int64'), 'AU57S': dtype('int64'), 'BC20S': dtype('int64'), 'IN01S': dtype('int64'), 'BI29S': dtype('int64'), 'BI34S': dtype('int64'), 'BC30': dtype('float64'), 'DS35': dtype('float64'), 'HI12S': dtype('int64'), 'G070': dtype('int64'), 'FR06S': dtype('int64'), 'BR57S': dtype('int64'), 'S207A': dtype('int64'), 'BC01': dtype('int64'), 'PB35S': dtype('int64'), 'CV01': dtype('int64'), 'BC97B': dtype('int64'), 'G220A': dtype('int64'), 'AT36S': dtype('int64'), 'G236S': dtype('int64'), 'OF01S': dtype('int64'), 'CV06': dtype('int64'), 'HI29S': dtype('int64'), 'PNC_Payment_Rate_Score_ARGUS': dtype('float64'), 'G209S': dtype('int64'), 'PB20': dtype('float64'), 'MT57S': dtype('int64'), 'RE101S': dtype('int64'), 'G024': dtype('int64'), 'HI30S': dtype('int64'), 'MT45': dtype('int64'), 'BR01S': dtype('int64'), 'BC14': dtype('int64'), 'S209S': dtype('int64'), 'Filler8': dtype('float64'), 'G220B': dtype('int64'), 'G228S': dtype('int64'), 'MT48': dtype('int64'), 'AGG905': dtype('float64'), 'MT20S': dtype('int64'), 'G066S': dtype('int64'), 'Zip4': dtype('float64'), 'RE29S': dtype('int64'), 'IN28': dtype('int64'), 'G211S': dtype('int64'), 'AU36S': dtype('int64'), 'UR33': dtype('int64'), 'BKC53': dtype('int64'), 'BI12S': dtype('int64'), 'CV08': dtype('int64'), 'CV25': dtype('int64'), 'LM12S': dtype('int64'), 'AGG107': dtype('int64'), 'G210S': dtype('int64'), 'BKC328': dtype('int64'), 'HI35S': dtype('int64'), 'BR02S': dtype('int64'), 'FC07S': dtype('int64'), 'G315S': dtype('int64'), 'prior_dist_BK_flag': dtype('int64'), 'G061S': dtype('int64'), 'BC110S': dtype('int64'), 'FI29S': dtype('int64'), 'G051': dtype('float64'), 'IN34S': dtype('int64'), 'PB14': dtype('int64'), 'ST57S': dtype('int64'), 'UR21': dtype('float64'), 'BI01S': dtype('int64'), 'AU32S': dtype('int64'), 'MT32S': dtype('int64'), 'HI21S': dtype('int64'), 'IN25S': dtype('int64'), 'BI08': dtype('int64'), 'BC09': dtype('int64'), 'DS11': dtype('int64'), 'AU34S': dtype('int64')}

def create_thin_flag(df):
    at20 = df.AT20.copy()
    df['thin_flag'] = (np.where(at20 >= 0, 1, 0)) & (np.where(at20 <= 24, 1, 0))
    return df


def create_prior_dist_bk_flag(df):
    ftb16 = df.FTB16.copy()
    df['prior_dist_BK_flag'] = (np.where(ftb16 >= 0, 1, 0)) & (np.where(ftb16 <= 48, 1, 0))
    return df


def read_data(csv_path):
    df = (dd.read_csv(csv_path,dtype=meta_dtypes )
          .pipe(create_thin_flag)
          .pipe(create_prior_dist_bk_flag)
          .rename(columns={'Vantage_Score_3': 'VTG03TOT', 'EADS51_Score': 'EADS51'}))

    return df

def pipeline(df):
    return (df.pipe(create_thin_flag)
          .pipe(create_prior_dist_bk_flag)
          .rename(columns={'Vantage_Score_3': 'VTG03TOT', 'EADS51_Score': 'EADS51'})
          .pipe(score)
          .pipe(bc31_at35_cut)
          .pipe(bc31_at35_bc21_cut)
          .pipe(apply_all_cuts)
         )

from collections import OrderedDict
meta_final_types = OrderedDict([('TU_Edit_Sequence_Number', dtype('int64')),
              ('BC31', dtype('float64')),
              ('AT35', dtype('float64')),
              ('BC21', dtype('float64')),
              ('CEMP09', dtype('int64')),
              ('scored_trio_uncalibrated', dtype('float32')),
              ('scored_ts_uncalibrated', dtype('float32')),
              ('BC31_AT35_cut', dtype('O')),
              ('BC31_AT35_BC21_cut', dtype('O')),
              ('all_cuts', dtype('O'))])

meta_final = pd.DataFrame(columns = meta_final_types.keys())
for each in meta_final_types.keys():
    meta_final[each] = meta_final[each].astype(meta_final_types[each])
meta_final.dtypes

df= dd.read_csv('../january_campaign/kt*.txt', dtype=meta_dtypes, blocksize=None)

dask.set_options(get=dask.multiprocessing.get)

(df.map_partitions(pipeline, meta=meta_final)
   .drop(['BC31_AT35_cut','BC31_AT35_BC21_cut','all_cuts'], axis=1)).visualize()

%%time 
from dask.diagnostics import ProgressBar
with ProgressBar():
    scored=(df.map_partitions(pipeline, meta=meta_final).compute(num_workers=3))

scored.to_csv('scored_full_january_file_v2.csv', float_format='%.6f',index=False)